In [ ]:
# Importação das bibliotecas
import pandas as pd

from app.infra.sqlalchemy.config.database import engine

In [ ]:
# Mapeamento dos dias da semana
dias_da_semana = {
    '2': 'Segunda',
    '3': 'Terça',
    '4': 'Quarta',
    '5': 'Quinta',
    '6': 'Sexta',
    '7': 'Sábado'
}

# Mapeamento dos horários das aulas
horarios_aulas = {
    'M1': '08h00 às 08h55',
    'M2': '08h55 às 09h50',
    'M3': '10h00 às 10h55',
    'M4': '10h55 às 11h50',
    'M5': '12h00 às 12h55',

    'T1': '12h55 às 13h50',
    'T2': '14h00 às 14h55',
    'T3': '14h55 às 15h50',
    'T4': '16h00 às 16h55',
    'T5': '16h55 às 17h50',
    'T6': '18h00 às 18h55',

    'N1': '19h00 às 19h50',
    'N2': '19h50 às 20h40',
    'N3': '20h50 às 21h40',
    'N4': '21h40 às 22h30',
    # 'M1': '08h00 às 09h50',
    # 'M2': '08h00 às 09h50',
    # 'M3': '10h00 às 11h50',
    # 'M4': '10h00 às 11h50',
    # 'M5': '12h00 às 13h50',

    # 'T1': '12h00 às 13h50',
    # 'T2': '14h00 às 15h50',
    # 'T3': '14h00 às 15h50',
    # 'T4': '16h00 às 17h50',
    # 'T5': '16h00 às 17h50',
    # 'T6': '18h00 às 18h55',

    # 'N1': '19h00 às 20h40',
    # 'N2': '19h00 às 20h40',
    # 'N3': '20h50 às 22h30',
    # 'N4': '20h50 às 22h30',
}

In [ ]:
def separa_horario(horario):
    """Função que separa os dias e horários de cada disciplina"""

    horario = list(horario)

    for i in range(len(horario)):
        turno = horario[i]
        if turno.isalpha():
            # Em 246T45, os 3 primeiros dígitos representam os dias e os 3 últimos o horário da aula
            dias, horas = horario[:i], horario[i + 1:]
            break

    # Separa os dias da aula
    dias = [dias_da_semana[d] for d in dias]

    # Busca os horários
    horas = [f'{turno}{hora}' for hora in horas]

    hora_aula = [horarios_aulas[hora] for hora in horas]

    # Retorna uma tupla contendo uma lista dos dias e o horário da aula
    return (dias, hora_aula)

In [ ]:
def gera_grade_UnB(disciplinas: dict, pendentes: list) -> pd.DataFrame:
    """Função que gera a grade horária da UnB"""

    # Cria um dicionário para armazenar os dias e horários de cada disciplina
    dicio_horarios = {dia: {hora: [] for hora in horarios_aulas.values()}
                      for dia in dias_da_semana.values()}

    # Percorre as disciplinas
    for id_disciplina, (codigo, horarios, docente) in disciplinas.items():
        # Verifica se a diciplina não foi feita
        # if (feita):
        if (codigo in pendentes):
            # Percorre os horários da disciplina
            for horario in horarios:
                # Aplica a função que separa os dias e horários
                dias, horas = separa_horario(horario)

                # Percorre os dias que aquela disciplina é ministrada
                for dia in dias:
                    for hora in horas:
                        # Popula o dicionário
                        dicio_horarios[dia][hora].append(
                            f'{id_disciplina}')

    # Cria uma tabela com os dias da semana
    tabela_semana = {dia: [] for dia in dias_da_semana.values()}

    # Cria uma lista ordenada com os horários de aula
    lista_horarios = sorted(
        set(hora for valor in dicio_horarios.values() for hora in valor.keys()))

    # Percorre os dias da semana
    for dia in dias_da_semana.values():
        # Percorre os horários da lista de horários
        for horario in lista_horarios:
            # Verifica se o horário está no dicionário
            if horario in dicio_horarios[dia]:
                # Se sim, adiciona a sigla da disciplina no respectivo dia da semana e horário
                tabela_semana[dia].append(
                    '/'.join(dicio_horarios[dia][horario]))

    # Cria um dataframe com a grade horária
    return pd.DataFrame(tabela_semana, index=[hora for hora in lista_horarios])

In [ ]:
sql_disciplinas = '''
SELECT
    *
FROM public.teste_grade_disciplinas
'''

lista_disciplinas = pd.read_sql(sql=sql_disciplinas, con=engine)

In [ ]:
disciplinas = {d['id_disciplina']: [d['codigo'], str(d['horarios']).split(
    '/'), d['docente']] for idx, d in lista_disciplinas.iterrows()}

In [ ]:
deptos = {}
disci = []

for idx, d in lista_disciplinas.iterrows():
    texto = f"{d['codigo']} - {d['nome']}"

    if texto not in disci:
        disci.append(texto)

    if d['depto'] not in deptos:
        deptos[d['depto']] = [texto]

    else:
        if texto not in deptos[d['depto']]:
            deptos[d['depto']].append(texto)

deptos = dict(sorted(deptos.items()))
disci.sort()

In [ ]:
pendentes = [
    # 3º
    # 'MAT0034', # Álgebra 1
    # 'MAT0053', # Cálculo Numérico
    # 4º
    'CIC0182', # Lógica Computacional 1
    # 5º
    'CIC0105', # Engenharia de Software
    # 6º
    'CIC0104', # Software Básico
    'CIC0161', # Autômatos e Computabilidade
    'CIC0203', # Computação Experimental
    # 'CIC0205', # Fundamentos de Sistemas Operacionais
    # 7º
    'CIC0189', # Projeto e Análise de Algoritmos
    # 'CIC0201', # Segurança Computacional
    'CIC0204', # Compiladores
    # 8º
    'CIC0200', # Metodologia Científica
    'CIC0249', # Trabalho de Graduação 1
    # 9º
    'CIC0250', # Trabalho de Graduação 2
]

In [ ]:
grade = gera_grade_UnB(disciplinas=disciplinas, pendentes=pendentes)
display(grade)

In [ ]:
grade.to_csv('Grade UnB.csv', sep=',', encoding='UTF-8', index=True)